# Particle Filters

### Moving Robot

In [1]:
class Map:
    def __init__(self, world_size, landmarks):
        self.world_size = world_size
        self.landmarks = landmarks

In [2]:
from math import *
import random

class Robot:
    def __init__(self, world_size):
        self.x = random.random()
        self.y = random.random()
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0
        self.turn_noise = 0.0
        self.sense_noise = 0.0
        
    def set(self, new_x, new_y, new_orientation, world_size):
        if new_x < 0 or new_x >= world_size:
            raise (ValueError, 'X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise (ValueError, 'Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise (ValueError, 'Orientation must be in [0..2pi]')
        
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
        
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        self.forward_noise = float(new_f_noise)
        self.turn_noise = float(new_t_noise)
        self.sense_noise = float(new_s_noise)
        
    def sense(self, landmarks):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    def move(self, turn, forward, world_size):
        if forward < 0:
            raise (ValueError, 'Robot cant move backwards')
            
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size
        y %= world_size
        self.set(x, y, orientation, world_size)
    
    def Gaussian(self, mu, sigma, x):
        return exp(-(mu - x) ** 2) / (sigma ** 2 / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    def measurement_prob(self, measurement, landmarks):
        prob = 1.0
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))

In [3]:
def eval(r, p):
    sum = 0.0
    for i in range(len(p)):
        dx = (p[i].x - r.x + world_size/2.0) % world_size - (world_size / 2.0)
        dy = (p[i].y - r.y + world_size/2.0) % world_size - (world_size / 2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))

In [4]:
landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0
gmap = Map(world_size, landmarks)

In [5]:
robot = Robot(gmap.world_size)
robot.set(30.0, 50.0, (pi/2), gmap.world_size)
robot.move(-(pi/2), 15, gmap.world_size)
print(robot.sense(gmap.landmarks))

robot.move(-(pi/2), 10, gmap.world_size)
print(robot.sense(gmap.landmarks))

[39.05124837953327, 46.09772228646444, 39.05124837953327, 46.09772228646444]
[32.01562118716424, 53.150729063673246, 47.16990566028302, 40.311288741492746]


### Importance Weight

In [41]:
robot = Robot(gmap.world_size);
robot.move(0.1, 5.0, gmap.world_size)
Z = robot.sense(gmap.landmarks)
print("Actual distance(between Robot and Landmarks):\n", Z, "\n")
print("Robot position:\n", robot, "\n")

N = 1000
particles = []
for i in range(N):
    p = Robot(gmap.world_size)
    p.set_noise(0.05, 0.05, 5.0)
    particles.append(p)
    
p2 = []
for p in particles:
    p.move(0.1, 5.0, gmap.world_size)
    p2.append(p)    
particles = p2

importance_weights = []
for p in particles:
    importance_weights.append(p.measurement_prob(Z, gmap.landmarks));

importance_weights_n = []
sigma = sum(importance_weights)
for weight in importance_weights:
    importance_weights_n.append(weight / sigma);

Actual distance(between Robot and Landmarks):
 [109.02085173438357, 24.190596410250123, 79.59786951454012, 78.3256678000277] 

Robot position:
 [x=97.921 y=96.247 orient=4.0802] 



### Resampling

In [45]:
p3 = []
idx = int(random.random() * N)
beta = 0.0
w_max = max(importance_weights_n)

for i in range(N):
    beta += random.random() * 2.0 * w_max
    while importance_weights_n[idx] < beta:
        beta -= importance_weights_n[idx]
        idx = (idx + 1) % N
    p3.append(particles[idx])
particles = p3

print("Position of Particles which represent the prediction of Robot position:")
for i in range(10):
    print(particles[i])

Position of Particles which represent the prediction of Robot position:
[x=96.857 y=97.174 orient=3.7892]
[x=98.631 y=95.784 orient=4.4331]
[x=98.041 y=96.162 orient=4.1946]
[x=98.041 y=96.162 orient=4.1946]
[x=98.041 y=96.162 orient=4.1946]
[x=97.459 y=96.734 orient=4.0368]
[x=97.898 y=96.465 orient=4.1108]
[x=97.202 y=96.574 orient=4.0231]
[x=97.481 y=96.204 orient=4.0608]
[x=97.165 y=96.783 orient=3.9145]


In [ ]:
class ParticleFilter:
    def __init__(self, gmap):
        self.gmap = gmap
        
    def get_importance_weight(self):
        w = []
        return w